In [1]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
import pandas as pd


IMAGE_SIZE = 64 

def resize_image(image, height = IMAGE_SIZE, width = IMAGE_SIZE):
    top, bottom, left, right = (0,0,0,0)
    
    h, w, _ = image.shape
    longest_edge = max(h,w)
    
    # Add padding to shorter edge
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass 
    # RGB color
    BLACK = [0,0,0]
    # add padding
    # cv2.BORDER_CONSTANT: fill colors which same as edge
    constant = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value = BLACK)
    return cv2.resize(constant, (height, width))


images = []
labels = []


def read_path(path_name):
    for dir_item in os.listdir(path_name): 
        full_path = os.path.abspath(os.path.join(path_name, dir_item))    
        if os.path.isdir(full_path):
            read_path(full_path)
        else: 
            if dir_item.endswith('.jpg'):
                image = cv2.imread(full_path)
                if image is None: 
                    pass
                else:
	                image = resize_image(image, IMAGE_SIZE, IMAGE_SIZE)
	                images.append(image)
	                labels.append(path_name)
    return images, labels

def load_dataset(path_name):
    images,labels = read_path(path_name)
    
    images = np.array(images, dtype='float') 
#    print(images.shape) 
    labels = np.array([str(label) for label in labels])
    temp = []
    for file_path in labels:
        name = file_path.split("\\")[-1]
        temp.append(name)
    names = pd.DataFrame(temp, columns=['name'])
    le = LabelEncoder().fit(names['name'])
    labels = le.transform(names['name'])
    df = pd.DataFrame(columns=['name', 'encode'])
    df['name'] = names['name'].values
    df['encode'] = labels
    df.drop_duplicates(keep='first', inplace=True)
    return images, labels, df


In [2]:
import random
import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D,BatchNormalization
from keras.optimizers import SGD
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import os
import sys

# from face_data import load_dataset, resize_image, IMAGE_SIZE
import cv2



def load(path_name, img_rows = IMAGE_SIZE, img_cols = IMAGE_SIZE, img_channels = 3, nb_classes = 2):

    images, labels, names = load_dataset(path_name)

    train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size = 0.2, random_state = random.randint(0, 100))

    if K.image_data_format == 'channel_first': 
        train_images = train_images.reshape(train_images.shape[0],img_channels, img_rows, img_cols)
        test_images = test_images.reshape(test_images.shape[0],img_channels, img_rows, img_cols)
        input_shape = (img_channels, img_rows, img_cols)
    else:
        train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, img_channels)
        test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, img_channels)
        input_shape = (img_rows, img_cols, img_channels)

    print(train_images.shape[0], 'train samples')
    print(test_images.shape[0], 'test samples')
        
#		 #One hot encode
#        train_labels = keras.utils.to_categorical(train_labels, labels.size)
#        test_labels = keras.utils.to_categorical(test_labels, labels.size)

        #normalization
    train_images /= 255
    test_images /= 255
        
    return train_images, test_images, train_labels, test_labels, input_shape, names

Using TensorFlow backend.


In [15]:
from keras.layers.pooling import _Pooling2D
from keras.utils import conv_utils
from keras.engine.base_layer import InputSpec
class LBPH2D(_Pooling2D):
    def __init__(self, pool_size=(2, 2), strides=None, padding='valid',
                 data_format=None, **kwargs):
        super(_Pooling2D, self).__init__(**kwargs)
        if strides is None:
            strides = pool_size
        self.pool_size = conv_utils.normalize_tuple(pool_size, 2, 'pool_size')
        self.strides = conv_utils.normalize_tuple(strides, 2, 'strides')
        self.padding = conv_utils.normalize_padding(padding)
        self.data_format = K.normalize_data_format(data_format)
        self.input_spec = InputSpec(ndim=4)
    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            rows = input_shape[2]
            cols = input_shape[3]
        elif self.data_format == 'channels_last':
            rows = input_shape[1]
            cols = input_shape[2]
        rows = conv_utils.conv_output_length(rows, self.pool_size[0],
                                             self.padding, self.strides[0])
        cols = conv_utils.conv_output_length(cols, self.pool_size[1],
                                             self.padding, self.strides[1])
        if self.data_format == 'channels_first':
            return (input_shape[0], input_shape[1], rows, cols)
        elif self.data_format == 'channels_last':
            return (input_shape[0], rows, cols, input_shape[3])


    def _pooling_function(self, inputs, pool_size, strides,
                          padding, data_format):
        output = pooling(inputMap=inputs,poolSize=pool_size,poolStride=strides,mode='lbph')
        return output

In [17]:
import numpy as np
import random
import tensorflow as tf

def pooling(inputMap,poolSize=4,poolStride=2,mode='max'):
    print(poolSize)
    print(type(inputMap))
    print(type(poolStride))
    """INPUTS:
              inputMap - input array of the pooling layer
              poolSize - X-size(equivalent to Y-size) of receptive field
              poolStride - the stride size between successive pooling squares
       
       OUTPUTS:
               outputMap - output array of the pooling layer
               
       Padding mode - 'edge'
    """
    # inputMap sizes
    poolStride = poolStride[0]
    print(type(poolStride))
    poolSize = poolSize[0]
    print(type(poolSize))
    inn = np.shape(inputMap)
    print(inn)
    in_row = int(inn[1])
    print(type(in_row))
    in_col = int(inn[2])
    in_layers = int(inn[3])
    print(in_row)
    print(in_col)
    # outputMap sizes
    out_row = int(np.floor((in_row-poolSize)/poolStride)+1)
    out_col = int(np.floor((in_col-poolSize)/poolStride)+1)
    row_remainder,col_remainder = np.mod(in_row,poolStride),np.mod(in_col,poolStride)
    outputMap = np.zeros((out_row,out_col,in_layers))
    # padding
    #temp_map = tf.pad(inputMap, tf.constant([[0, poolSize-row_remainder], [0, poolSize-row_remainder]],[0,0]))
    #temp_map = inputMap.eval()
    #print(inputMap.shape)
    #x = tf.placeholder(tf.float32, shape=(？,64,64,3))
    sess = tf.Session()
    if inputMap.shape.as_list()[0]==None:
        return 
    with sess.as_default():
        print(tf.slice(inputMap,[1,0,0],[None,in_row,in_col,in_layers]))
    if mode == "max":
    # max pooling
        for r_idx in range(0,out_row):
            for c_idx in range(0,out_col):
                startX = c_idx * poolStride
                startY = r_idx * poolStride
                poolField = temp_map[0,startY:startY + poolSize, startX:startX + poolSize]
                poolOut = np.max(poolField)
                outputMap[r_idx,c_idx] = poolOut
           
    # retrun outputMap
    elif mode == "lbph":
        for l_idx in range(0,in_layers):
            
            for r_idx in range(0,out_row):
                for c_idx in range(0,out_col):
                    startX = c_idx * poolStride
                    startY = r_idx * poolStride
                    poolField = inputMap[startY:startY + poolSize, startX:startX + poolSize,l_idx]
                    print(poolField)
                    a = 0
                    if(poolSize%2==0):
                        if poolSize%2==0:
                            a = np.mean(poolField[int(in_row/2-1):int(in_row/2),int(in_col/2-1):int(in_col/2)])
                        else:
                            a = np.mean(poolField[int((in_row-1)/2),int(in_col/2-1):int(in_col/2)])
                    else:
                        if poolSize%2==0:
                            a = np.mean(poolField[int(in_row/2-1):int(in_row/2),int((in_col-1)/2)])
                        else:
                            a = poolField[int((in_row-1)/2),int((in_col-1)/2)]
                    length = poolSize*poolSize-(poolSize-2)*(poolSize-2)
                    array = np.zeros(length)
                    for x in range(poolSize-1):
                        s1 = poolField[poolSize-1,x]
                        s2 = poolField[poolSize-1-x,poolSize-1]
                        s3 = poolField[0,poolSize-1-x]
                        s4 = poolField[x,0]
                        array[x] = s1
                        array[x+poolSize-1] = s2
                        array[x+poolSize*2-2] = s3
                        array[x+poolSize*3-3] = s4
                    array_zeroone=np.where(array<a,0,1)
                    poolOut = 0
                    for x in range(length):
                        poolOut += array_zeroone[x]*np.power(2,length-1-x)
                    outputMap[r_idx,c_idx,l_idx] = poolOut
    return  outputMap
# 测试实例
#test = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
#test_result = pooling(test, 2,1, 'lbph')
#print(test_result)

In [5]:
def build_LBPmodel(x_train, y_train, input_shape, nb_classes = 34, batch_size = 128, nb_epoch = 30, data_augmentation = True, valid_data=None):
    
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3),strides=(1, 1), activation='relu', input_shape = input_shape))
    model.add(LBPH2D(pool_size=4,strides=1))
    model.add(Dropout(0.2))
  
    model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same'))
    model.add(LBPH2D(pool_size=4,strides=1))
    model.add(Dropout(0.2))
        
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same'))
    model.add(LBPH2D(pool_size=4,strides=1))
    model.add(Dropout(0.2))
		
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), activation='relu', padding='same'))
    model.add(LBPH2D(pool_size=4,strides=1))
    model.add(Dropout(0.2))
        
    model.add(Flatten())
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
        
    model.add(Dense(nb_classes, activation='softmax'))
        
#     file_path = "./model/"
#     file_path = "drive/INFO_6105/model"
#     file_dir = os.path.dirname(file_path)
#     if not os.path.exists(file_dir):
#         os.makedirs(file_dir)
        
    model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    if not data_augmentation:
        model.fit(x_train, y_train, 
                       batch_size = batch_size,
                        epochs = nb_epoch, 
                        shuffle = True)
    else:
        datagen = ImageDataGenerator(rotation_range = 20, 
                                         width_shift_range  = 0.2, 
                                         height_shift_range = 0.2, 
                                         horizontal_flip = True)                           
        model.fit_generator(datagen.flow(x_train, y_train,
                                                    batch_size = batch_size),
                                                    steps_per_epoch=len(x_train),
                                                    epochs = nb_epoch)
#     model.save('/content/gdrive/My Drive/INFO_6105/face/model/LBPNN.face.model.h5')
    return model

In [18]:
train_images, test_images, train_labels, test_labels, input_shape, names = load('./training-data/train')
model = build_LBPmodel(train_images, train_labels, input_shape)

13272 train samples
3318 test samples
(4, 4)
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tuple'>
<class 'int'>
<class 'int'>
(?, 62, 62, 32)
<class 'int'>
62
62


AttributeError: 'NoneType' object has no attribute '_keras_shape'